# 3x Барьерный метод

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [2]:
# ToDo Добавить разные atr для разных временных лимитов
# Пример
# tbm_5d: pt_sl_multipliers=[1.5, 1.5] (более узкие барьеры)
# tbm_10d: pt_sl_multipliers=[2, 2] (наша база)
# tbm_20d: pt_sl_multipliers=[3, 3] (более широкие барьеры)

In [3]:
# ==============================================================================
# ПУНКТ 3: ФОРМИРОВАНИЕ TBM
# ==============================================================================

def get_tbm_labels(prices, volatility, horizons, pt_sl_multipliers):
    """
    Основная, оптимизированная функция для разметки данных по методу Трех Барьеров.
    
    :param prices: pd.Series с ценами закрытия.
    :param volatility: pd.Series с дневной волатильностью (например, ATR).
    :param horizons: pd.Series с горизонтом (вертикальный барьер) для каждой точки.
    :param pt_sl_multipliers: list/tuple, например [2, 2] для pt=2*vol, sl=2*vol.
    :return: pd.Series с метками (+1, -1, 0).
    """
    # Преобразуем в numpy-массивы для максимальной скорости
    prices_arr = prices.to_numpy()
    volatility_arr = volatility.to_numpy()
    horizons_arr = horizons.to_numpy()
    labels_arr = np.zeros_like(prices_arr, dtype=np.int8)

    pt_levels = prices_arr + pt_sl_multipliers[0] * volatility_arr
    sl_levels = prices_arr - pt_sl_multipliers[1] * volatility_arr

    for i in tqdm(range(len(prices_arr)), desc="TBM Labeling", leave=False, delay=0.1):
        horizon = int(horizons_arr[i])
        if horizon <= 0:
            continue
        
        # Определяем срез будущих цен
        end_idx = i + 1 + horizon
        path = prices_arr[i+1 : end_idx]
        if len(path) == 0:
            continue

        # Векторизованные проверки на касание барьеров
        pt_hits = np.where(path >= pt_levels[i])[0]
        sl_hits = np.where(path <= sl_levels[i])[0]

        # Находим индекс первого касания
        pt_first_hit_idx = pt_hits[0] if len(pt_hits) > 0 else -1
        sl_first_hit_idx = sl_hits[0] if len(sl_hits) > 0 else -1

        # Определяем, какой барьер был достигнут первым
        if pt_first_hit_idx != -1 and sl_first_hit_idx != -1:
            labels_arr[i] = 1 if pt_first_hit_idx <= sl_first_hit_idx else -1
        elif pt_first_hit_idx != -1:
            labels_arr[i] = 1
        elif sl_first_hit_idx != -1:
            labels_arr[i] = -1
        # Если касаний не было, метка остается 0 (вертикальный барьер)

    return pd.Series(labels_arr, index=prices.index)




In [4]:
# ==============================================================================
# ОБНОВЛЕННАЯ ФУНКЦИЯ-ОБЕРТКА add_tbm
# ==============================================================================

def add_tbm_after_cusum(df, pt_sl_multipliers=[2, 2], volatility_col='atr_14'):
    """
    Добавляет TBM-метки только для строк, отмеченных CUSUM-фильтром.
    """
    print("Добавление меток TBM только для CUSUM-событий...")
    
    if volatility_col not in df.columns or 'cusum_event' not in df.columns:
        raise ValueError(f"ОШИБКА: Необходимые колонки ('{volatility_col}', 'cusum_event') не найдены.")

    # 1. Создаем горизонты для TBM, как и раньше
    
    horizons_to_add = [2, 3, 5, 7, 10, 15, 20, 25, 30, 40, 60]
    for h in horizons_to_add:
        df[f'tbm_{h}d_horizon'] = h
    
    df['tmp_day_of_week'] = pd.to_datetime(df['Date']).dt.dayofweek
    df['tbm_friday_horizon'] = 4 - df['tmp_day_of_week']
    df.loc[df['tbm_friday_horizon'] < 0, 'tbm_friday_horizon'] = 0
    
    tbm_horizon_cols = [col for col in df.columns if 'tbm_' in col and '_horizon' in col]
    final_labels_list = []

    grouped = df.groupby('Ticker')

    for ticker, group in tqdm(grouped, desc="Обработка тикеров для TBM"):
        group = group.sort_values('Date')
        volatility = group[volatility_col]
        
        # Находим индексы, где сработал CUSUM-фильтр
        event_indices = group[group['cusum_event'] != 0].index
        
        ticker_labels = pd.DataFrame(index=group.index)
        for col in tbm_horizon_cols:
            horizons = group[col].copy()
            
            # Обнуляем горизонты для всех строк, КРОМЕ тех, где было CUSUM-событие
            horizons.loc[~horizons.index.isin(event_indices)] = 0
            
            # Запускаем TBM. Она будет работать быстро, т.к. будет считать только для ненулевых горизонтов
            labels = get_tbm_labels(group['Close'], volatility, horizons, pt_sl_multipliers)
            
            # Заполняем NaN там, где не было CUSUM-события, чтобы потом их отбросить
            labels.loc[horizons == 0] = np.nan
            
            ticker_labels[col.replace('_horizon', '')] = labels
            
        final_labels_list.append(ticker_labels)
        
    all_labels_df = pd.concat(final_labels_list)
    df = df.join(all_labels_df)
    
    # Удаляем вспомогательные колонки
    df.drop(columns=tbm_horizon_cols + ['tmp_day_of_week'], inplace=True, errors='ignore')
    
    print("Добавление TBM завершено.")
    return df




# def add_tbm_to_features(df, pt_sl_multipliers=[2, 2], volatility_col='atr_14'):
#     """
#     Функция-обертка для добавления в DataFrame колонок с метками TBM.
#     """
#     print("Добавление меток TBM...")
    
#     if volatility_col not in df.columns:
#         raise ValueError(f"ОШИБКА: Колонка с волатильностью '{volatility_col}' не найдена в DataFrame.")

#     # 1.3.1: Создаем горизонты для TBM на фиксированный срок (5, 10, 20, ..., дней)
#     horizons_to_add = [2, 3, 5, 7, 10, 15, 20, 25, 30, 40, 60]
#     for h in horizons_to_add:
#         df[f'tbm_{h}d_horizon'] = h
    
#     # 1.3.2: Создаем горизонт для TBM до конца недели (до пятницы)
#     # day_of_week уже должен быть в df из предыдущего шага. 4 = Пятница.
#     df['tbm_friday_horizon'] = 4 - df['day_of_week']
#     df.loc[df['tbm_friday_horizon'] < 0, 'tbm_friday_horizon'] = 0 # В саму пятницу горизонт 0
    
#     # --- Выполняем разметку для каждого тикера и типа горизонта ---
#     tbm_horizon_cols = [col for col in df.columns if 'tbm_' in col and '_horizon' in col]
#     final_labels_list = []

#     grouped = df.groupby('Ticker')

#     for ticker, group in tqdm(grouped, desc="Обработка тикеров для TBM"):
#         # Убедимся, что для разметки данные отсортированы по дате
#         group = group.sort_values('Date')
        
#         volatility = group[volatility_col]
        
#         ticker_labels = pd.DataFrame(index=group.index)
#         for col in tbm_horizon_cols:
#             horizons = group[col]
#             labels = get_tbm_labels(group['Close'], volatility, horizons, pt_sl_multipliers)
#             # Создаем финальное имя колонки (e.g., 'tbm_5d')
#             ticker_labels[col.replace('_horizon', '')] = labels
            
#         final_labels_list.append(ticker_labels)
        
#     # Соединяем результаты разметки с основным DataFrame
#     all_labels_df = pd.concat(final_labels_list)
#     df = df.join(all_labels_df)
    
#     # Удаляем вспомогательные колонки с горизонтами
#     df.drop(columns=tbm_horizon_cols, inplace=True)
    
#     print("Добавление TBM завершено.")


#     return df

In [5]:
# ==============================================================================
# ОСНОВНОЙ СКРИПТ
# ==============================================================================

data_folder = "../data/"
input_filename = 'moex_with_cusum_events.csv'
final_dataset_filename = 'moex_final_dataset.csv'

# --- ШАГ 1: Загрузка данных с признаками ---
print(f"Загрузка данных с признаками из: {data_folder + input_filename}")
try:
    data_with_features = pd.read_csv(data_folder + input_filename)
    # Важно: преобразуем колонку с датой в правильный формат
    data_with_features['Date'] = pd.to_datetime(data_with_features['Date'])
    print("Данные успешно загружены.")
except FileNotFoundError:
    print(f"ОШИБКА: Файл не найден. Убедитесь, что скрипт feature_engineering.py был запущен.")
    exit()

# --- ШАГ 2: Добавление TBM меток ---
# Коэффициенты [2, 2] означают:
# Take Profit = Close + 2 * ATR
# Stop Loss  = Close - 2 * ATR
final_data = add_tbm_after_cusum(data_with_features, pt_sl_multipliers=[2, 2])

print("\n--- Финальный DataFrame ---")
print(final_data.info())

# Посмотрим на результат
print("\nПример финальных данных:")
print(final_data.tail())

final_data.drop("cusum_event", axis=1, inplace=True)

# --- ШАГ 3: Сохранение финального датасета ---
print(f"\nСохранение финального датасета в файл: {data_folder + final_dataset_filename}")
final_data.to_csv(data_folder + final_dataset_filename, index=False)

print("\nСкрипт разметки TBM выполнен успешно!")
print(f"Файл '{final_dataset_filename}' готов для обучения моделей.")

Загрузка данных с признаками из: ../data/moex_with_cusum_events.csv
Данные успешно загружены.
Добавление меток TBM только для CUSUM-событий...


Обработка тикеров для TBM:   0%|          | 0/46 [00:00<?, ?it/s]

TBM Labeling:   0%|          | 1/330 [00:00<00:52,  6.27it/s]

TBM Labeling:   0%|          | 1/353 [00:00<00:36,  9.74it/s]

TBM Labeling:   0%|          | 1/357 [00:00<00:37,  9.48it/s]

TBM Labeling:   0%|          | 1/370 [00:00<00:43,  8.52it/s]

TBM Labeling:   0%|          | 1/319 [00:00<00:32,  9.79it/s]

TBM Labeling:   0%|          | 1/319 [00:00<00:39,  8.02it/s]

Добавление TBM завершено.

--- Финальный DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15493 entries, 0 to 15492
Columns: 268 entries, Date to tbm_friday
dtypes: datetime64[ns](1), float64(230), int64(36), object(1)
memory usage: 31.7+ MB
None

Пример финальных данных:
            Date Ticker     Open     High      Low    Close  Volume  \
15488 2025-08-18   VSMO  35000.0  35460.0  33300.0  35220.0  8151.0   
15489 2025-08-25   VSMO  33500.0  33560.0  31440.0  32500.0  9746.0   
15490 2025-09-01   VSMO  35120.0  35380.0  34600.0  34940.0  2917.0   
15491 2025-09-08   VSMO  34760.0  35600.0  34260.0  34940.0  3997.0   
15492 2025-09-15   VSMO  33720.0  34180.0  32920.0  33120.0  1479.0   

       sarimax_pred_1d_to_today  sarimax_pi_width_rel_1d  \
15488                  2.198139                 0.097454   
15489                  3.422163                 0.104874   
15490                  0.587187                 0.111777   
15491                 -0.724236              

In [6]:
final_data

,Date,Ticker,Open,High,Low,Close,Volume,sarimax_pred_1d_to_today,sarimax_pi_width_rel_1d,sarimax_pred_3d_to_today,...,tbm_5d,tbm_7d,tbm_10d,tbm_15d,tbm_20d,tbm_25d,tbm_30d,tbm_40d,tbm_60d,tbm_friday
0,2019-03-19,AFKS,9.889,9.895,9.830,9.845,3170800.0,-0.044602,0.060704,-0.114192,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-03-26,AFKS,9.723,9.850,9.686,9.770,7156200.0,-0.567535,0.061299,-0.614071,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,2019-04-02,AFKS,9.356,9.600,9.180,9.310,29374900.0,0.107234,0.060160,-0.036326,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
3,2019-04-09,AFKS,9.243,9.270,9.131,9.146,10023700.0,0.373753,0.061039,0.404753,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
4,2019-04-16,AFKS,9.068,9.100,9.021,9.079,4995200.0,-0.640005,0.061603,-0.680522,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15488,2025-08-18,VSMO,35000.000,35460.000,33300.000,35220.000,8151.0,2.198139,0.097454,2.317652,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15489,2025-08-25,VSMO,33500.000,33560.000,31440.000,32500.000,9746.0,3.422163,0.104874,3.535282,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15490,2025-09-01,VSMO,35120.000,35380.000,34600.000,34940.000,2917.0,0.587187,0.111777,0.715780,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15491,2025-09-08,VSMO,34760.000,35600.000,34260.000,34940.000,3997.0,-0.724236,0.103858,-0.710558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
